In [1]:
import CellularAutomata
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets

In [2]:
def fireFocus(M, N, i, j, size):
    focus = np.zeros((M, N))
    focus[i-size:i+size, j-size:j+size] = np.ones((size, size)) 
    return focus

In [3]:
def plotState(states, t):
    plt.imshow(states[t], origin='lower')
    plt.show()

In [4]:
def plotStates(t=0):
    slider = widgets.IntSlider(value=0, min=0, max=len(states)-1, step=1, description='Time step:', 
                               continuous_update=False, readout=True, readout_format='d')
    
    widgets.interact(plotState, states=widgets.fixed(states), t=slider)

In [37]:
# Testing
M = 101
N = 101
initial = fireFocus(M, N, 50, 50, 1)
rule = 2
#neighborhood = 'vonneumann'
neighborhood = 'moore'
times = 100

In [38]:
CA = CellularAutomata.CA(M, N, initial, rule, neighborhood)
states = CA.propagate(times)

In [39]:
plotStates()

interactive(children=(IntSlider(value=0, continuous_update=False, description='Time step:', max=99), Output()), _dom_classes=('widget-interact',))